# Intro to pygeodesic

Shows how to use pygeodesic to compute geodesic distances on the flat surface mesh provided with the original C++ code.

Uses VTK to visualise the mesh, the geodesic distance and the geodesic path.

In [1]:
# Imports
import pygeodesic
import pygeodesic.geodesic as geodesic
import numpy as np
import vtk
from vtk_helpers import *

In [2]:
pygeodesic.__version__

'0.1.0'

## Compute geodesic distance and path between 2 points

In [3]:
# Read the mesh to get the points and faces of the mesh
filename = r'data/flat_triangular_mesh.txt'
result = geodesic.read_mesh_from_file(filename)
if result:
    points, faces = result

In [4]:
# Initialise the PyGeodesicAlgorithmExact class instance
geoalg = geodesic.PyGeodesicAlgorithmExact(points,faces)

In [5]:
# Get the distance and the path between the source and target points
sourceIndex = 25
targetIndex = 97
distance, path = geoalg.geodesicDistance(sourceIndex, targetIndex)

In [6]:
# The geodesic distance between the source and target points
distance

1.697056274847714

In [7]:
# The geodesic path, represented by a number of 3D points along the path
path

array([[ 3.1,  0.8,  0. ],
       [ 3. ,  0.7,  0. ],
       [ 2.9,  0.6,  0. ],
       [ 2.8,  0.5,  0. ],
       [ 2.7,  0.4,  0. ],
       [ 2.6,  0.3,  0. ],
       [ 2.5,  0.2,  0. ],
       [ 2.4,  0.1,  0. ],
       [ 2.3,  0. ,  0. ],
       [ 2.2, -0.1,  0. ],
       [ 2.1, -0.2,  0. ],
       [ 2. , -0.3,  0. ],
       [ 1.9, -0.4,  0. ]])

## Compute geodesic distance between source point and all other points

In [8]:
distances, best_source = geoalg.geodesicDistances(np.array([sourceIndex,]))

In [9]:
distances

array([0.72111026, 0.56568542, 0.4472136 , 0.4       , 0.4472136 ,
       0.56568542, 0.72111026, 0.89442719, 1.07703296, 1.26491106,
       1.45602198, 0.63245553, 0.4472136 , 0.28284271, 0.2       ,
       0.28284271, 0.4472136 , 0.63245553, 0.82462113, 1.0198039 ,
       1.21655251, 1.41421356, 0.6       , 0.4       , 0.2       ,
       0.        , 0.2       , 0.4       , 0.6       , 0.8       ,
       1.        , 1.2       , 1.4       , 0.63245553, 0.4472136 ,
       0.28284271, 0.2       , 0.28284271, 0.4472136 , 0.63245553,
       0.82462113, 1.0198039 , 1.21655251, 1.41421356, 0.72111026,
       0.56568542, 0.4472136 , 0.4       , 0.4472136 , 0.56568542,
       0.72111026, 0.89442719, 1.07703296, 1.26491106, 1.45602198,
       0.84852814, 0.72111026, 0.63245553, 0.6       , 0.63245553,
       0.72111026, 0.84852814, 1.        , 1.16619038, 1.34164079,
       1.52315462, 1.        , 0.89442719, 0.82462113, 0.8       ,
       0.82462113, 0.89442719, 1.        , 1.13137085, 1.28062

## Visualise path using VTK

In [10]:
# Create actors
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
path_actor = createPolyLineActor(path, color=(1,1,1))
point_actors = [createSphereActor(points[indx], radius=0.03) for indx in [sourceIndex, targetIndex]]

In [11]:
# Add "distances" to polydata_actor to visualise distance contour from source point
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(distances))
polydata_actor.GetMapper().SetScalarRange([distances.min(),distances.max()])

In [12]:
# Show VTK render window
v = Viewer()
v.addActors([polydata_actor, path_actor, *point_actors])
v.show()

## Compute geodesic distance between 2 source points and all other points

In [13]:
sources = np.array([25,100,])
distances, best_source = geoalg.geodesicDistances(sources)

In [14]:
distances, best_source

(array([0.72111026, 0.56568542, 0.4472136 , 0.4       , 0.4472136 ,
        0.56568542, 0.72111026, 0.89442719, 1.07703296, 1.26491106,
        1.45602198, 0.63245553, 0.4472136 , 0.28284271, 0.2       ,
        0.28284271, 0.4472136 , 0.63245553, 0.82462113, 1.0198039 ,
        1.21655251, 1.41421356, 0.6       , 0.4       , 0.2       ,
        0.        , 0.2       , 0.4       , 0.6       , 0.8       ,
        1.        , 1.2       , 1.4       , 0.63245553, 0.4472136 ,
        0.28284271, 0.2       , 0.28284271, 0.4472136 , 0.63245553,
        0.82462113, 1.0198039 , 1.21655251, 1.41421356, 0.72111026,
        0.56568542, 0.4472136 , 0.4       , 0.4472136 , 0.56568542,
        0.72111026, 0.89442719, 1.07703296, 1.26491106, 1.45602198,
        0.82462113, 0.72111026, 0.63245553, 0.6       , 0.63245553,
        0.72111026, 0.84852814, 1.        , 1.16619038, 1.34164079,
        1.52315462, 0.63245553, 0.6       , 0.63245553, 0.72111026,
        0.82462113, 0.89442719, 1.        , 1.13

## Visualise geodesic distance to closest source using VTK

Color mesh by the distance to theclosest source

In [15]:
# Create actors
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
point_actors = [createSphereActor(p,radius=0.03) for p in points[sources]]

In [16]:
# Add "distances" to polydata_actor to visualise distance contour from source point
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(distances))
polydata_actor.GetMapper().SetScalarRange([distances.min(),distances.max()])

In [17]:
# Show VTK render window
v = Viewer()
v.addActors([polydata_actor, *point_actors])
v.show()

## Visualise best source for all points using VTK

Color mesh by the index of the closest source

In [18]:
# Create actors
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
point_actors = [createSphereActor(p,radius=0.03) for p in points[sources]]

In [19]:
# Add "distances" to polydata_actor to visualise distance contour from source point
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(best_source))

In [20]:
# Show VTK render window
v = Viewer()
v.addActors([polydata_actor, *point_actors])
v.show()